In [1]:
import pandas as pd

## Get information for sewer

In [2]:
block = pd.read_csv("~/Dropbox/CDS-2019-AlbanyHub/ToDatabase/full_blockgroup.csv") #block group/census tract
#Number of addresses with sewer data, per block group
address_counts = pd.read_csv("~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/queryresults_sewer/address_count_per_blockgroup.csv")
#average (unit of consumption) for houses with sewer data, per block group
avg_kwh = pd.read_csv("~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/queryresults_sewer/avg_swr_per_blockgroup.csv")
#average (unit of consumption) for houses where housing project occured and sewer data exists, per block group
#also includes number of addresses where housing project occured and sewer data exists, per block group
avg_kwh_block = pd.read_csv("~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/queryresults_sewer/avg_swr_per_blockgroup_housingprojects.csv")

In [3]:
#Crop census data and rename it to be in line with other files
df = (block[["total_vacdata", "block_group", "tract"]]).rename(columns = {"tract":"Tract","block_group":"BlockGroup"})

In [4]:
#Merge these files into one dataframe, preserving all important information
df_full = df.merge(address_counts, on = ["Tract", "BlockGroup"], how = "inner")
#df_full = df_full.merge(address_count_block, on = ['Tract', "BlockGroup"], how = "left")
df_full = df_full.merge(avg_kwh, on = ['Tract', "BlockGroup"], how = "left")
df_full = df_full.merge(avg_kwh_block, on = ['Tract', "BlockGroup"], how = "left")

In [5]:
#replace NaN from left join with 0
df_full = df_full.fillna(0)

In [6]:
#change name to reflect contents
df_full = df_full.rename(columns = {"address_count_x":"utilities_address_count", "address_count_y":"projects_address_count"})

In [7]:
#create column: the number of addresses that were not part of housing projects
df_full["nonproj_address_count"] = df_full["utilities_address_count"] - df_full["projects_address_count"]

In [8]:
#Convert counts to percent of houses
df_full["proj_pct"] = df_full["projects_address_count"] / df_full["utilities_address_count"] * 100

In [9]:
df_full["nonproj_pct"] = 100 - df_full["proj_pct"]

In [10]:
#change name to reflect content
df_full = df_full.rename(columns = {"mean_swr":"project_mean_swr", "avg_swr":"nonproj_mean_swr", 'avg_charge':'nonproj_mean_charge', 'mean_charge':'proj_mean_charge'}).drop("numrecords", axis = 1)

In [11]:
df_full = df_full.rename(columns = {"total_vacdata":"total_properties"})

In [12]:
#rearrange columns for more logical order
cols = list(df_full)
cols.insert(0, cols.pop(cols.index('Tract')))
cols.insert(1, cols.pop(cols.index("BlockGroup")))
cols.insert(5, cols.pop(cols.index("nonproj_address_count")))
cols.insert(4, cols.pop(cols.index("projects_address_count")))
cols.insert(6, cols.pop(cols.index("proj_pct")))
cols.insert(7, cols.pop(cols.index("nonproj_pct")))
df_full = df_full.loc[:, cols]

In [13]:
df_full.head()

,Tract,BlockGroup,total_properties,utilities_address_count,projects_address_count,nonproj_mean_swr,proj_pct,nonproj_pct,nonproj_address_count,nonproj_mean_charge,project_mean_swr,proj_mean_charge
0,100,1,433,137,7.0,5.179593,5.109489,94.890511,130.0,24.290165,3.660855,19.977027
1,100,2,175,106,1.0,4.990617,0.943396,99.056604,105.0,23.056171,3.285289,18.972892
2,100,3,397,335,7.0,5.285681,2.089552,97.910448,328.0,24.487208,3.052323,18.239529
3,100,4,938,218,4.0,7.931480,1.834862,98.165138,214.0,37.566103,4.673691,22.528912
4,100,5,493,269,5.0,5.300894,1.858736,98.141264,264.0,24.659227,5.462084,25.396139


In [14]:
#Create totals column
sum_props = df_full["total_properties"].sum()#number of properties, total according to census data
sum_utilities = df_full["utilities_address_count"].sum()#number of addresses with residential sewer data
sum_projects = df_full["projects_address_count"].sum()#number of housing project addresses with residential sewer data
sum_nonproj = df_full["nonproj_address_count"].sum()#number of non-housing project addresses with residential sewer data

In [15]:
df_full = df_full.append({"total_properties" : sum_props, "utilities_address_count" : sum_utilities,
                         "projects_address_count" : sum_projects, "nonproj_address_count" : sum_nonproj}, ignore_index =  True)

In [16]:
#manually enter values for totals row
df_full.loc[65, "Tract"] = "Totals"
df_full.loc[65, "proj_pct"] = sum_projects/sum_utilities*100
df_full.loc[65, "nonproj_pct"] = sum_nonproj/sum_utilities*100
#below are generated from sql queries in albany-housing-analytics/sql/Table-queries-sewer
df_full.loc[65, "project_mean_swr"] = 9.92
df_full.loc[65, "nonproj_mean_swr"] = 5.39
df_full.loc[65, "proj_mean_charge"] = 47.2205628340595
df_full.loc[65, "nonproj_mean_charge"] = 25.647916971282

In [17]:
df_full

,Tract,BlockGroup,total_properties,utilities_address_count,projects_address_count,nonproj_mean_swr,proj_pct,nonproj_pct,nonproj_address_count,nonproj_mean_charge,project_mean_swr,proj_mean_charge
0,100,1.0,433.0,137.0,7.0,5.179593,5.109489,94.890511,130.0,24.290165,3.660855,19.977027
1,100,2.0,175.0,106.0,1.0,4.990617,0.943396,99.056604,105.0,23.056171,3.285289,18.972892
2,100,3.0,397.0,335.0,7.0,5.285681,2.089552,97.910448,328.0,24.487208,3.052323,18.239529
3,100,4.0,938.0,218.0,4.0,7.931480,1.834862,98.165138,214.0,37.566103,4.673691,22.528912
4,100,5.0,493.0,269.0,5.0,5.300894,1.858736,98.141264,264.0,24.659227,5.462084,25.396139
5,200,1.0,352.0,247.0,11.0,4.802298,4.453441,95.546559,236.0,23.121201,4.614119,22.789646
6,200,2.0,170.0,161.0,5.0,4.958329,3.105590,96.894410,156.0,23.624569,4.114848,21.283446
7,200,3.0,560.0,251.0,26.0,5.148065,10.358566,89.641434,225.0,24.287164,5.374796,24.872320
8,200,4.0,244.0,161.0,7.0,6.978661,4.347826,95.652174,154.0,35.991885,3.099119,18.238901
9,400,1.0,738.0,417.0,2.0,5.084804,0.479616,99.520384,415.0,24.091406,5.115270,23.818721


In [73]:
df_full.to_csv("~/Dropbox/CDS-2019-AlbanyHub/full_table_sewer.csv", index=False)